# MD in Colab

## 0. Connect to Google Drive & Set working environment

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
assert gpu_info.find('command not found') <0, "Please change to GPU runtime!"

#@markdown Runtime will be restarted shortly. Please wait.

!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...


In [ ]:
#@markdown Enter the name of the working directory within your Google Drive. <br/>
#@markdown Please authorize the Colab to get access to your Google Drive. MD Trajectory files are large, and Colab sessions are notorious for their instabilty.

import subprocess

print('Installing all required python packages...', end='')
subprocess.run(['mamba', 'install', '-c', 'pytorch', '-c', 'nvidia', '-c', 'conda-forge',
                'pytorch', 'pytorch-cuda=12.1', 'openmm=8', 'openmm-torch', 'openmm-ml',
                'openmm-xtb', 'nnpops', 'espaloma', 'openmmforcefields', 'scikit-learn',
                'scipy', 'ipywidgets=7', 'nglview', 'parmed', 'rdkit', 'pdbfixer', 'openbabel',
                'plotly', 'mdtraj', 'lxml', '--yes'])
subprocess.run(['pip', 'install', 'mace-torch', '--yes'])
subprocess.run(['apt-get', 'install', 'python3-openbabel'])
subprocess.run(['pip', 'install', '--no-deps', 'plip'])
print('done.')

from pdbfixer import PDBFixer
from openmm import *
from openmm.app import *
from openmm.unit import *
from openmmxtb import XtbForce
from openmmml import MLPotential
from openff.toolkit import Molecule
from openmmforcefields.generators import GAFFTemplateGenerator
from openmmforcefields.generators import SMIRNOFFTemplateGenerator
from openmmforcefields.generators import EspalomaTemplateGenerator
from openmm.app.internal import customgbforces
from rdkit import Chem
from rdkit.Chem import AllChem
import io
import os
import locale
import sys
import glob
import copy
import warnings
import threading
import numpy as np
import parmed as pmd
import nglview as nv
from time import sleep
from tqdm.notebook import tqdm
from IPython.display import display, clear_output
import mdtraj as md
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
from collections import Counter
import openbabel
from plip.structure.preparation import PDBComplex
from plip.exchange.report import BindingSiteReport
from scipy.spatial.transform import Rotation
from sklearn.covariance import EmpiricalCovariance
from collections import Counter
import ipywidgets as widgets
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'colab'
from google.colab import drive, output, files
output.enable_custom_widget_manager()


def navigator(working_directory)->str:
    work = "/content/drive/MyDrive/"+working_directory

    if not os.path.isdir(work):
        print(f'No directory named {working_directory} in your Google Drive.')
        os.mkdir(work)
        print(f'Created a new directory named {working_directory} in your Google Drive.')
        print('You may now proceed to the cell# 1-1.')

    elif os.path.exists(work+'/production_final.xtc'):
        print(f'There is a finished MD simulation in the directory {working_directory}.')
        print('You can extend your production run at cell# 3-3 if you want.')
        print('To continue analysis, go to cell# 4-1.')
        print('To calculate binding free energy, go to cell# 5-1.')

    elif os.path.exists(work+'/production_0.xtc'):
        print('Seems like you were performing production run.')
        print('Go to cell# 3-3.')

    elif os.path.exists(work+'/npt_0.xtc'):
        print('Seems like you were performing NPT equilibration.')
        print('Go to cell# 3-2.')

    elif os.path.exists(work+'/nvt_0.xtc'):
        print('Seems like you were performing NVT equilibration.')
        print('Go to cell# 3-1.')

    elif os.path.exists(work+'/start.xml'):
        print('You have created your simulation object, but never started it.')
        print('Go to cell# 3-1.')

    elif os.path.exists(work+'/starting_structure.pdb'):
        print('You have prepared the starting structure, but did not create the simulation object.')
        print('Go to cell# 2-1.')

    else:
        print(f'There is a directory named {working_directory} in you Google Drive, but it is empty.')
        print('Proceed to cell# 1-1.')

    work += '/'
    return work


def read_settings(work:str)->dict:
    """
    Read the parameters from the settings.txt file.
    """
    if not os.path.exists(work+'settings.txt'):
        return None

    with open(work+'settings.txt', 'r') as f:
        settings = {}
        for line in f.readlines():
            if line.strip().startswith('Forcefield'):
                settings['forcefield'] = line.split(":")[1].strip()
            elif line.strip().startswith('Watermodel'):
                settings['watermodel'] = line.split(":")[1].strip()
            elif line.strip().startswith('Constraints'):
                settings['constraints'] = line.split(":")[1].strip()
            elif line.strip().startswith('Rigidwater'):
                settings['rigidwater'] = bool(line.split(":")[1].strip())
            elif line.strip().startswith('Hydrogen mass'):
                settings['hydrogen mass'] = float(line.split(":")[1].strip())*amu
            elif line.strip().startswith('Precision'):
                settings['precision'] = line.split(":")[1].strip()
            elif line.strip().startswith('Temperature'):
                settings['temperature'] = (float(line.split(":")[1].strip())+273.15)*kelvin
            elif line.strip().startswith('Pressure'):
                settings['pressure'] = float(line.split(":")[1].strip())*bar
            elif line.strip().startswith('Time step'):
                settings['stepsize'] = float(line.split(":")[1].strip())*femtosecond
            elif line.strip().startswith('Ionic strength'):
                settings['ionic strength'] = float(line.split(":")[1].strip())*molar
            elif line.strip().startswith('Periodic Boundary Condition'):
                settings['PBC'] = True if line.split(":")[1].strip()=="True" else False
            elif line.strip().startswith('Padding'):
                settings['padding'] = float(line.split(":")[1].strip())*nanometer
            elif line.strip().startswith('Cation'):
                settings['cation'] = line.split(":")[1].strip()
            elif line.strip().startswith('Anion'):
                settings['anion'] = line.split(":")[1].strip()
            elif line.strip().startswith('Save interval'):
                settings['save interval'] = int(line.split(":")[1].strip())*picosecond
            elif line.strip().startswith('Restraint'):
                settings['restraint'] = int(line.split(":")[1].strip())

    return settings


def read_non_standard_residues_list(work)->list:
    """
    Read the nonstandard residue list from the non_standard_residues.txt file.
    """
    if not os.path.exists(work+'non_standard_residues.txt'):
        return []

    with open(work+'non_standard_residues.txt', 'r') as f:
        nsres = []
        for line in f.readlines():
            if not line.startswith('#'):
                data = line.strip().split( )
                if len(data) > 1:
                    if data[1] == 'ligand':
                        lig = True
                    elif data[1] == 'residue':
                        lig = False
                    nsres.append({'name':data[0], 'lig':lig, 'ff':data[2]})
                else:
                    nsres.append({'name':data[0]})
    return nsres

def determine_forcefield(forcefield, watermodel)->tuple:
    """
    Assign the macromolecular forcefield & water model from the choice.
    """
    if forcefield == "Amber14":
        ff = ForceField('amber14-all.xml')
        if watermodel == "(explicit) TIP3P":
            ff.loadFile('amber14/tip3p.xml')
            water = 'tip3p'
        elif watermodel == "(explicit) SPC/E":
            ff.loadFile('amber14/spce.xml')
            water = 'spce'
        elif watermodel == "(explicit) TIP4P-Ew":
            ff.loadFile('amber14/tip4pew.xml')
            water = 'tip4pew'
        elif watermodel == "(explicit) TIP5P":
            print("ERROR : AMBER forcefield does not support TIP5P water model!")
            print("Please choose other options.")
            return None, ""
        elif watermodel == "(explicit) OPC":
            ff.loadFile('amber14/opc.xml')
            water = 'tip4pew'
        elif watermodel == "(explicit) OPC3":
            ff.loadFile('amber14/opc3.xml')
            water = 'tip3p'
        elif watermodel == "(implicit) HCT (igb=1)":
            ff.loadFile('implicit/hct.xml')
            water = ""
        elif watermodel == "(implicit) OBC1 (igb=2)":
            ff.loadFile('implicit/obc1.xml')
            water = ""
        elif watermodel == "(implicit) OBC2 (igb=5)":
            ff.loadFile('implicit/obc2.xml')
            water = ""
        elif watermodel == "(implicit) GBn (igb=7)":
            ff.loadFile('implicit/gbn.xml')
            water = ""
        elif watermodel == "(implicit) GBn2 (igb=8)":
            ff.loadFile('implicit/gbn2.xml')
            water = ""
        else:
            print('Error in reading AMBER forcefield parameters.')
            return None, ""

    elif forcefield == "CHARMM36":
        if watermodel == "(explicit) TIP3P":
            ff = ForceField('charmm36.xml', 'charmm36/water.xml')
            water = 'tip3p'
        elif watermodel == "(explicit) SPC/E":
            ff = ForceField('charmm36.xml', 'charmm36/spce.xml')
            water = 'spce'
        elif watermodel == "(explicit) TIP4P-Ew":
            ff = ForceField('charmm36.xml', 'charmm36/tip4pew.xml')
            water = 'tip4pew'
        elif watermodel == "(explicit) TIP5P":
            ff = ForceField('charmm36.xml', 'charmm36/tip5p.xml')
            water = 'tip5p'
        elif watermodel == "(explicit) OPC":
            print("ERROR : CHARMM forcefield does not support OPC water models!")
            print("Please choose other options.")
            return None, ""
        elif watermodel == "(explicit) OPC3":
            print("ERROR : CHARMM forcefield does not support OPC water models!")
            print("Please choose other options.")
            return None, ""
        elif watermodel == "(implicit) HCT (igb=1)":
            ff = ForceField('charmm36.xml', 'implicit/hct.xml')
            water = ''
        elif watermodel == "(implicit) OBC1 (igb=2)":
            ff = ForceField('charmm36.xml', 'implicit/obc1.xml')
            water = ''
        elif watermodel == "(implicit) OBC2 (igb=5)":
            ff = ForceField('charmm36.xml', 'implicit/obc2.xml')
            water = ''
        elif watermodel == "(implicit) GBn (igb=7)":
            ff = ForceField('charmm36.xml', 'implicit/gbn.xml')
            water = ''
        elif watermodel == "(implicit) GBn2 (igb=8)":
            ff = ForceField('charmm36.xml', 'implicit/gbn2.xml')
            water = ''
        else:
            print("Error in reading the CHARMM forcefield parameters")
            return None, ""

    else:
        return None, ""

    return ff, water


class SimulationStream(io.StringIO):
    """
    StringIO's child class to capture stdout simulation output.
    """
    def __init__(self):
        super().__init__()
        self.string = ""

    def write(self, string):
        self.string += string

    def read(self, size=-1)->str:
        if size < 0:
            result = self.string
            self.string = ""
        else:
            result = self.string[:size]
            self.string = self.string[size:]
        return result


def apply_restraints(work, simulation, restraint):
    """
    Apply a harmonic positional restraint potential to the heavy atoms.
    """
    if restraint > 0:
        print('Applying positional harmonic restraints...', end='')
        posres = CustomExternalForce("k*periodicdistance(x, y, z, x0, y0, z0)^2")
        posres.addGlobalParameter("k", restraint)
        posres.addPerParticleParameter("x0")
        posres.addPerParticleParameter("y0")
        posres.addPerParticleParameter("z0")

        pdb = PDBFile(work+'minimized.pdb')
        heavy = []
        for i, a in enumerate(pdb.topology.atoms()):
            if a.element.symbol != 'H':
                heavy.append(i)

        positions = simulation.context.getState(getPositions=True).getPositions(asNumpy=True)
        for i in range(np.shape(positions)[0]):
            if i in heavy:
                posres.addParticle(i, positions[i])
        simulation.system.addForce(posres)
        print('done.')
    return simulation



def restore_simulation_from_state(work, statename, step, stage):
    """
    Restore a simulation object from a state file.
    """
    s = read_settings(work)
    pdb = PDBFile(work+'minimized.pdb')
    system = XmlSerializer.deserialize(open(work+'system.xml').read())
    integrator = LangevinMiddleIntegrator(s['temperature'], 1/picosecond, s['stepsize'])
    platform = Platform.getPlatformByName('CUDA')
    properties = {'Precision': s['precision']}
    simulation = Simulation(pdb.topology, system, integrator, platform, properties)
    simulation.context.setState(XmlSerializer.deserialize(open(statename).read()))
    change_flag = False # When the context is changed, reinitialize it
    if stage in ['npt', 'production'] and s['PBC']:
        barostat = MonteCarloBarostat(s['pressure'], s['temperature'])
        simulation.system.addForce(barostat)
        change_flag = True
    if stage != 'production' and stage != 'start':
        simulation = apply_restraints(work, simulation, s['restraint'])
        change_flag = True
    if change_flag:
        simulation.context.reinitialize(preserveState=True)

    simulation.context.setStepCount(step)
    time = step*s['stepsize']/picosecond
    simulation.context.setTime(time)

    return simulation


def recall_simulation(work):
    """
    Recall a simulation object from the working directory.
    """
    if len(glob.glob(work+'production_step_*.xml')) > 0:
        stage = "production"
        print('Resuming production run...', end='')
        for f in sorted(glob.glob(work+'production_step_*.xml'), reverse=True):
            step = int(f.split('.')[-2].split('_')[-1])
            simulation = restore_simulation_from_state(work, f, step, 'production')
            break

    elif len(glob.glob(work+'npt_step_*.xml')) > 0:
        stage = "npt"
        print('Resuming NPT equilibration...', end='')
        for f in sorted(glob.glob(work+'npt_step_*.xml'), reverse=True):
            step = int(f.split('.')[-2].split('_')[-1])
            simulation = restore_simulation_from_state(work, f, step, 'npt')
            break

    elif len(glob.glob(work+'nvt_step_*.xml')) > 0:
        stage = "nvt"
        print('Resuming NVT equilibration...', end='')
        for f in sorted(glob.glob(work+'nvt_step_*.xml'), reverse=True):
            step = int(f.split('.')[-2].split('_')[-1])
            simulation = restore_simulation_from_state(work, f, step, 'nvt')
            break

    elif len(glob.glob(work+'start.xml')) > 0:
        stage = "start"
        print('Starting the simulation...', end='')
        simulation = restore_simulation_from_state(work, work+'start.xml', 0, 'start')

    else:
        print('Error in reading the simulation object.')
        return None, ""

    print('done.')
    return simulation, stage


def run_simulation(work, stage, simulation, stream, length, save_period, backup_period):
    """
    Run the simulation object for length picoseconds.
    """

    s = read_settings(work)

    runtime = 0.0
    time = simulation.context.getTime().value_in_unit(picosecond)

    while runtime < length:
        step = simulation.context.getStepCount()
        log = StateDataReporter(stream, 50, step=True, potentialEnergy=True,
                                temperature=True, speed=True, time=True,
                                density=s['PBC'])

        i = 0
        for f in glob.glob(work+stage+'_*.xtc'):
            i += 1

        save_freq = int(save_period/s['stepsize'])
        xtc = XTCReporter(work+stage+f'_{i}.xtc', save_freq, False, s['PBC'])

        simulation.reporters.clear()
        simulation.reporters.append(log)
        simulation.reporters.append(xtc)

        for i in range(backup_period*60):
            simulation.runForClockTime(1*second)
            newStep = simulation.context.getStepCount()
            now = simulation.context.getTime().value_in_unit(picosecond)
            runtime = now - time
            if runtime >= length:
                break
            lines = stream.read()
            with open(work+stage+'_log.txt', 'a') as f:
                f.write(lines)
            line = lines.split('\n')
            if len(line) < 2:
                continue
            data = line[-2].split(',')
            print('', end='\r')
            if len(data) < 6:
                speed = float(data[4])
                density = ''
            else:
                speed = float(data[5])
                density = f'Density: {float(data[4]):.4f} g/mL, '
            if stage != 'production':
                T = f'Time: {float(data[1]):.1f} ps, '
            else:
                T = f'Time: {float(data[1])/1000:.2f} ns, '

            if speed > 0:
                d, hs = divmod((length - runtime)/speed/1000*24, 24)
                h, mins = divmod(hs*60, 60)
                min, secs = divmod(mins*60, 60)
                sec = int(secs)
                remaining = f'{str(int(d))+":" if d > 0 else ""}{int(h):0>2}:{int(min):0>2}:{sec:0>2} left, '
            else:
                remaining = '--:--:-- left, '

            print(f'{remaining}Step: {data[0]}, {T}Temperature: {(float(data[3])-273.15):.2f} C, {density}Speed: {speed:.1f} ns/day', end='')

        state = simulation.context.getState(getPositions=True, getVelocities=True)
        with open(work+stage+'_step_'+str(newStep)+'.xml', 'w') as f:
            f.write(XmlSerializer.serialize(state))
        os.remove(work+stage+'_step_'+str(step)+'.xml')

    print('\nSimulation finished!')
    return


drive.mount('/content/drive')
working_directory = "test_caf1" #@param {type:"string"}
work = navigator(working_directory)

## 1. PDB Input Preparation

In [ ]:
#@title 1-1. Add missing atoms & Remove unrecognized molecules

assert 'work' in globals(), "Run cell# 0 to set working directory."

#@markdown Enter the 4-letter PDB ID to fetch PDB file.<br/>
#@markdown If you want to upload your PDB file, leave it as empty.
PDB_ID = "6X18" #@param {type:"string"}
#@markdown For ligands/non-standard residues that are defined in [RCSB Chemical Component Dictionary](https://www.wwpdb.org/data/ccd),
#@markdown enter the residue name below. <br/>For more than one non-standard residues, separate them using commas without spaces.
#@markdown <br/>**The residue names you entered must match with the ones present in the PDB file!**
non_standard_residues = "" #@param {type:"string"}
#@markdown For ligands/non-standard residues that are not defined on the RCSB Dictionary, run the next cell to upload the SDF file.<br/>
#@markdown <br/> Check the box below to remove the water molecules present in your PDB.
remove_crystallographic_water = True #@param {type:"boolean"}

amino_acids = ['ALA', 'CYS', 'ASP', 'GLU', 'PHE', 'GLY', 'HIS', 'ILE', 'LYS', 'LEU',
               'MET', 'ASN', 'PRO', 'GLN', 'ARG', 'SER', 'THR', 'VAL', 'TRP', 'TYR']
nucleic_acids = ['A', 'T', 'G', 'C', 'U', 'I', 'DA', 'DT', 'DG', 'DC', 'DI']

if non_standard_residues == "":
    nsres = []
else:
    nsres = non_standard_residues.split(',')

with open(work+'non_standard_residues.txt', 'w') as f:
    f.write('# User-defined Non-standard Residues\n')
    for res in nsres:
        f.write(res.upper()+'\n')

if PDB_ID == "":
    print("Upload your PDB file.")
    pdbfile = files.upload()
    try:
        fixer = PDBFixer(filename=next(iter(pdbfile)))
    except:
        print('Error in reading the input pdbfile.')
else:
    try:
        fixer = PDBFixer(pdbid=PDB_ID.upper())
    except:
        print('Error in fetching the pdb file from the RCSB.')

print('Adding missing atoms...', end='')
fixer.findMissingResidues()
terminal_missing_res = []
chains = list(fixer.topology.chains())
for key in fixer.missingResidues.keys():
    if key[1] == 0 or key[1] == len(list(chains[key[0]].residues())):
        terminal_missing_res.append(key)
for key in terminal_missing_res:
    del fixer.missingResidues[key]
fixer.findMissingAtoms()
fixer.addMissingAtoms()

model = Modeller(fixer.topology, fixer.positions)
res_to_del = []
for res in model.topology.residues():
    if remove_crystallographic_water:
        if res.name not in amino_acids+nucleic_acids+nsres:
            res_to_del.append(res)
    else:
        if res.name not in (amino_acids+nucleic_acids+nsres+['HOH']):
            res_to_del.append(res)

model.delete(res_to_del)

print('done.')

with open(work+"fixed.pdb", 'w') as f:
    PDBFile.writeFile(model.topology, model.positions, f)

custom_info = {}

view1 = nv.NGLWidget()
view1._set_size('750px','500px')
view1.add_component(nv.FileStructure(work+"fixed.pdb"))
view1.add_licorice()
display(view1)

Adding missing atoms...done.


NGLWidget()

In [ ]:
#@title (Optional) Upload SDF files for Custom Ligands/Residues
#@markdown For ligands/non-standard residues not defined on RCSB Chemical Component Dictionary, <br/>
#@markdown run this cell to upload SDF file(s). <br/>

# assert 'work' in globals(), "Please run the cell #0 to set working directory."
# assert len(nsres) > 0, "No non-standard residues entered in the above cell!"

print("Upload your ligand SDF file.")
ligfiles = files.upload()

lignames = list(ligfiles.keys())
lig_select = widgets.Dropdown(options=lignames, value=lignames[0], description='Custom residue')
view0 = nv.NGLWidget()
view0._set_size('400px','300px')

for lig in lignames:
    custom_info[lig] = {}
custom_info[lig_select.value]['id'] = view0.add_component(nv.FileStructure(lig_select.value))

res_select = widgets.Dropdown(options=nsres, value=nsres[0], description='Choose residue name:')
custom_info[lig_select.value]['name'] = ""

def update_mol2(change): # callback function to interactively update viewer
    view0.remove_component(custom_info[change.old]['id'])
    custom_info[change.new]['id'] = view0.add_component(nv.FileStructure(change.new))
    view0.center()
    custom_info[change.new]['name'] = "" # res_select.value

def update_name(change):
    custom_info[lig_select.value]['name'] = ""

lig_select.observe(update_mol2, names='value')
# res_select.observe(update_name, names='value')

print('Assign residue name to the structures of the residues you uploaded.')

display(lig_select, view0, "")

IndexError: list index out of range

In [ ]:
#@title 1-2. Add hydrogen atoms to the PDB file

assert 'work' in globals(), "Please run the cell# 0 to set working directory."
assert os.path.exists(work+'fixed.pdb'), "Please run the cell# 1-1 first."

#@markdown Choose a method to add hydrogen atoms to your system : <br/>
#@markdown * PDB2PQR - Uses PropKa to protonate systems without ligands.
#@markdown * REDUCE - Works for ligands too, but only for physiological pH.
#@markdown * OpenBabel - Can automatically protonate any system, but often buggy
#@markdown * PDBFixer - Only works for standard residues
#@markdown * Custom - Upload your PDB file with hydrogens (use external tools like
#@markdown [DelPhipKa](http://compbio.clemson.edu/pka_webserver/),
#@markdown [H++](http://newbiophysics.cs.vt.edu/H++/),
#@markdown [PypKa](https://pypka.org/run-pypka/), etc.

protonation_method = "REDUCE" #@param ['PDB2PQR', 'REDUCE', 'OpenBabel', 'PDBFixer', 'Custom']

#@markdown Choose the reference pH for the determination of protonation state.
pH = 6 #@param {type:"slider", min:0.0, max:14.0, step:0.1}

for key, value in custom_info.items():
    subprocess.run(f'cp {key} {work}{value["name"]}.sdf', shell=True)


def protonate_w_pdbfixer(work, removeH, pH)->None:
    """
    Add missing hydrogen atoms to the PDB file using PDBFixer.
    """
    print('Adding hydrogens using PDBFixer...', end='')
    pdb = PDBFile(work+'fixed.pdb')
    model = Modeller(pdb.topology, pdb.positions)

    if removeH:
        Hs = []
        for a in model.topology.atoms():
            if a.element.symbol == 'H':
                Hs.append(a)
        model.delete(Hs)

    try:
        model.addHydrogens(pH=pH)
        with open(work+'starting_structure.pdb', 'w') as f:
            PDBFile.writeFile(model.topology, model.positions, f)
    except Exception as e:
        print(e)
    print('done.')
    return


def protonate_manually(work)->None:
    """
    Enable uploading of custom PDB file.
    """
    print('Here is your fixed pdb file.')
    files.download(work+'fixed.pdb')
    sleep(10)
    print('Now upload the pdb file with all the hydrogens.')
    fixed_wH = files.upload()
    filename = next(iter(fixed_wH))
    subprocess.run(f'cp -rf {filename} {work}starting_structure.pdb', shell=True)
    return


def protonate_w_openbabel(work, nsres, pH)->None:
    """
    Add missing hydrogen atoms to the PDB file using OpenBabel.
    Since OpenBabel names all the hydrogen as 'H', we need to rename them.
    """
    print('Adding hydrogens using OpenBabel...', end='')

    # save atom names before OpenBabel butchers them
    pdb = PDBFile(work+'fixed.pdb')
    nsatoms = []
    for res in pdb.topology.residues():
        if res.name in nsres:
            atoms = []
            for a in res.atoms():
                atoms.append(a.name)
            nsatoms.append({'name':res.name, 'chain':res.chain.id, 'index':res.id, 'atoms':atoms})

    locale.getpreferredencoding = lambda: "UTF-8"
    subprocess.run(['obabel', f'{work}fixed.pdb', '-O', 'tmp.pdb', '-xk', '-p', f'{pH}'])

    with open('tmp.pdb', 'r') as f: # just number Hs so that OpenMM can discriminate them
        lines = f.readlines()
        chain = ''
        resi = -100
        headlines = ''
        bodylines = []
        taillines = ''
        headflag = True
        for line in lines:
            if line.startswith('ATOM') or line.startswith('HETATM'):
                headflag = False
                if line[17:20] == 'UNL': # ligands/ncAAs
                    if line[21] != chain: # reset index
                        chain = line[21]
                        resi = -100
                    if int(line[22:26]) > resi:
                        resi = int(line[22:26])
                        count = 0 # 0-based index of nsres
                        Hcount = 1 # 1-based index of Hs in the residue
                    if line[77] == 'H':
                        name = 'H'+str(Hcount)
                        Hcount += 1
                    else:
                        name = nsatoms[resi-1]['atoms'][count]
                        count += 1
                    resname = nsatoms[resi-1]['name']
                    chainid = nsatoms[resi-1]['chain']
                    resindex = nsatoms[resi-1]['index']
                    newinfo = f'{name:^4} {resname:>3} {chainid}{resindex:>4}'
                elif line[77] == 'H': # element symbol
                    if line[21] != chain: # chain ID
                        chain = line[21]
                        resi = -100
                    if int(line[22:26]) > resi:
                        resi = int(line[22:26])
                        Hcount = 1
                    name = 'H'+str(Hcount) # count number of Hs in a residue
                    Hcount += 1
                    newinfo = f'{name:^4}'+line[16:26]
                else:
                    newinfo = line[12:26]
                newline = line[:12]
                newline += newinfo
                newline += line[26:]
                bodylines.append(newline)
            elif headflag:
                headlines += line
            else:
                taillines += line

    with open('tmp2.pdb', 'w') as f:
        f.write(headlines)
        for line in sorted(bodylines, key=lambda x: (x[21], int(x[22:26]))):
            f.write(line)  # sort by chainID then residue index
        f.write(taillines)

    warnings.filterwarnings('ignore')
    pdb2 = PDBFile('tmp2.pdb')

    bonds_wH = []
    for bond in pdb2.topology.bonds():
        if bond[0].element.symbol == 'H':
            bonds_wH.append([bond[1], bond[0]])
        elif bond[1].element.symbol == 'H':
            bonds_wH.append([bond[0], bond[1]])
        else:
            continue

    counter = 4
    for bond in bonds_wH:
        multi = [b[0] for b in bonds_wH].count(bond[0]) # number of equivalent Hs
        bond[1].residue = bond[0].residue
        if counter > multi or counter == 0:
            counter = multi
        if bond[0].name == 'N':
            if multi == 3: # N-terminus
                bond[1].name = 'H'+str(counter)
                counter -= 1
            else:          # amide proton
                bond[1].name = 'H'
                counter -= 1
        else:
            if multi == 1:
                bond[1].name = 'H'+bond[0].name[1:]
                counter -= 1
            elif multi == 2: # AMBER atom naming scheme (HB2, HB3 connected to CB)
                if bond[0].element.symbol == 'C':
                    bond[1].name = 'H'+bond[0].name[1:]+str(counter+1)
                    counter -= 1
                else:
                    bond[1].name = 'H'+bond[0].name[1:]+str(counter)
                    counter -= 1
            elif multi == 3:
                bond[1].name = 'H'+bond[0].name[1:]+str(counter)
                counter -= 1

    with open(work+'starting_structure.pdb', 'w') as f:
        PDBFile.writeFile(pdb2.topology, pdb2.positions, f)
    print('done.')
    return



def protonate_w_pdb2pqr(work, removeH, pH)->None:
    """
    Add missing hydrogen atoms to the PDB file using PDB2PQR.
    """
    print('Adding hydrogens using PDB2PQR...', end='')

    if removeH:
        pdb = PDBFile(work+'fixed.pdb')
        model = Modeller(pdb.topology, pdb.positions)
        Hs = []
        for a in model.topology.atoms():
            if a.element.symbol == 'H':
                Hs.append(a)
        model.delete(Hs)

        with open('fixed-H.pdb', 'w') as f:
            PDBFile.writeFile(model.topology, model.positions, f)

        params = ' fixed-H.pdb fixed.pqr'

    else:
        params = f' {work}fixed.pdb fixed.pqr'


    params += f' --ff AMBER --ffout AMBER --pdb-output {work}starting_structure.pdb'
    params += f' --titration-state-method propka --with-ph {pH} --pH {pH} --drop-water'
    subprocess.run("pdb2pqr"+params, shell=True)
    print('done.')
    return



def get_parmed_atom_from_name(structure, res_top, name):
    for i, res in enumerate(structure.topology.residues()):
        if res.name == res_top.name and res.index == res_top.index:
            residue = structure.residues[i]
            break
    for atom in residue.atoms:
        if atom.name == name:
            return atom

def get_parmed_res_from_top(structure, res_top):
    for i, res in enumerate(structure.topology.residues()):
        if res.name == res_top.name and res.index == res_top.index:
            return structure.residues[i]

def find_coordinate(structure, atom):
    for i, a in enumerate(structure.atoms):
        if a is atom:
            return np.array([structure[i].xx, structure[i].xy, structure[i].xz])

def add_missed_proton(structure, residue, coords)->None:
    """
    Rarely, REDUCE program omits amide proton of some residues.
    This function adds them back.
    """
    H = pmd.Atom(name='H', atomic_number=1, type='H', mass=1.008, charge=0.0)

    pmd_res = get_parmed_res_from_top(structure, residue)

    structure.add_atom_to_residue(H, pmd_res)

    for i, a in enumerate(structure.atoms):
        if a is H:
            structure[i].xx = coords[0]
            structure[i].xy = coords[1]
            structure[i].xz = coords[2]

    structure.assign_bonds()

    print(f'Added a missing amide proton at {residue}!')

    return



def protonate_w_reduce(work, removeH)->None:
    """
    Add missing hydrogen atoms to the PDB file using REDUCE.
    """
    print('Warning: With REDUCE, the reference pH is fixed to 7.4!')
    print('Adding hydrogens using REDUCE...', end='')

    if removeH:
        Hflag = '-DROP_HYDROGENS_ON_ATOM_RECORDS '
    else:
        Hflag = ''
    subprocess.run(f'reduce -BUILD {Hflag}{work}fixed.pdb > reduced.pdb', shell=True)
    print('done.')

    struc = pmd.load_file("reduced.pdb")
    struc.assign_bonds()

    for chain in struc.topology.chains():
        for i, res in enumerate(chain.residues()):
            if i == 0 or res.name == 'PRO':
                C = get_parmed_atom_from_name(struc, res, 'C')
                O = get_parmed_atom_from_name(struc, res, 'O')
                continue
            N = get_parmed_atom_from_name(struc, res, 'N')

            # In most cases, N-H is antiparallel to C=O
            a_names = [atom.name for atom in res.atoms()]
            if 'N' in a_names and 'CA' in a_names and 'C' in a_names and 'O' in a_names and 'H' not in a_names:
                v_c = find_coordinate(struc, C) - find_coordinate(struc, O)
                pos_H = find_coordinate(struc, N) + 1.01*v_c/np.linalg.norm(v_c)
                add_missed_proton(struc, res, pos_H)

            C = get_parmed_atom_from_name(struc, res, 'C')
            O = get_parmed_atom_from_name(struc, res, 'O')

    with open(work+'starting_structure.pdb', 'w') as f:
        PDBFile.writeFile(struc.topology, struc.positions, f)
    return


#@markdown Check below box to ignore the hydrogen atoms currently present in your PDB.
ignore_current_Hs = False #@param {type:"boolean"}

if protonation_method == "PDBFixer":
    protonate_w_pdbfixer(work, ignore_current_Hs, pH)

elif protonation_method == "OpenBabel":
    print('Installing OpenBabel...', end='')
    subprocess.run(['mamba', 'install', '-c', 'conda-forge', 'openbabel'])
    print('done.')
    protonate_w_openbabel(work, nsres, pH)

elif protonation_method == "PDB2PQR":
    print('Installing PDB2PQR...', end='')
    subprocess.run(['pip', 'install', 'pdb2pqr'])
    print('done.')
    protonate_w_pdb2pqr(work, ignore_current_Hs, pH)

elif protonation_method == 'REDUCE':
    print('Installing REDUCE...', end='')
    subprocess.run(['mamba', 'install', '-c', 'bioconda', 'reduce'])
    print('done.')
    protonate_w_reduce(work, ignore_current_Hs)

else:
    protonate_manually(work)



#@markdown Select the center of focus to zoom in/out.<br/>
#@markdown You may also long-click an atom to center zoom and translate molecule with a right drag.

to_lookat = []

pdb = PDBFile(work+'starting_structure.pdb')

for res in pdb.topology.residues():
    if res.name in nsres:
        to_lookat.append(res)

# Interactively shows the protonated structure
select_res = widgets.Dropdown(options=['whole structure']+to_lookat, value='whole structure', description='Focus at', continuous_update=True)
view2 = nv.NGLWidget()
view2._set_size('750px','500px')
try:
    view2.add_component(nv.FileStructure(work+"starting_structure.pdb"))
except:
    print(f"While adding hydrogen atoms, an Error occurred in {protonation_method}.")

view2.add_licorice()
view2.center()

def center_of_view(change): # callback function for the update
    view2.center(f'{change.new.id} and :{change.new.chain.id}' if change.new != 'whole structure' else None)

select_res.observe(center_of_view, names='value')

display(select_res, view2)

print("The protonated structure also is saved at the 'starting_structure.pdb' in your working directory.")
print("Please double check the protonation state of all residues, especially for non-standard ones!")
print("Subsequent steps will fail with incorrectly assigned atoms.")

KeyError: 'name'